# Geoelektrische Inversion

Die geoelktrische Inversion hat das Ziel, ein Untergrundmodell (spez. elektrische Widerstände) zu erzeugen, welches möglichst gut die Messdaten (elektrische Widerstände) erklären kann. Dies geschieht im Rahmen einer **least-squares**-Anpassung:

$$\Psi_d = \sqrt{
\sum_i^N \left(\frac{
    d_i - f_i(\mathbf{m})
}{
\epsilon_i
}
\right)^2
},$$

mit:

* $N$: Anzahl der Datenpunkte
* $d_i$: der i-te Datenpunkt (Messwert)
* $f_i(\mathbf{m})$: Die **Vorwärtsantwort**, also die erwarteten Messwerte für ein
* $\mathbf{m}$: Untergrundmodell mit M spezischen Widerstandswerten (die räumliche Verteilung der Materialeigenschaften)
* $\epsilon_i$: der i-te Datenfehler. Wir formulieren unsere Datenfehler mit Hilfe eines Fehlermodelles, welchen den Datenfehler aus einer absoluten und einer relativen Komponente bestimmt:
    $$\epsilon_i = a \cdot R_i + b,$$
    wobei $R_i = d_i$ der i-te Messwert $[\Omega]$ ist. $a$ ist der relative Messwert, unten angegeben als Prozentwert 
    `tdm.crtomo_cfg["mag_rel"] = 1`, $b$ ist der absolute Messwert in $[\Omega]$, angeben im Code hier: `tdm.crtomo_cfg["mag_abs"] = 1e-3`.
* $\Psi_d$ wird der **Datenmisfit** genannt

Der Misfit ist auch das erste Kriterium zur Bewertung einer Inversion: Wenn alle Datenpunkte *im Mittel* innerhalb ihres Datenfehlers angepasst wurden, dann geht der Datenmisfit (in unserem Fall auch der **RMS**-Root mean square, genannt) gegen 1.

* Werte über 1 werden nicht im Rahmen der Fehler angepasst. Das Ergebnis ist **ünterfittet**
    * Ausreißer suchen (systematische Fehler) und entfernen
    * Alternativ: Falls Datenfehler geraten werden, systematisch verkleinern, bis RMS von 1 erreicht ist
* Werte unter 1 bedeuten, dass Rauschkomponenten gefittet werden. Das Ergebnis ist **überfittet**
    * Datenfehler in der Inversion größer machen
    
Ein weiteres Kriterium zur Bewertung von Inversionen ist das Bild an sich. Ist es zu inhomogen, deutet dies auf **Artefakte** hin, also künstliche Strukturen, welcher von der Inversion eingebaut wurden, um Rauschkomponenten zu erklären. Diese Strukturen sind klein (Elektrodenabstand und kleiner) und treten oft als Dipolstrukturen mit nahe beieinander liegenden großen und kleinen Werten auf.

## Wichtige Nebenbenwerkung zu Misfits (Stichwort: Unterbestimmheit und Regularisierung)

Das geoelektrische Problem ist ein **gemischt-bestimmtes (mixed-determined)** Problem. Das heißt, dass manche Modellparameter genau oder überbestimmt sind, während andere unterbestimmt sind. Für überbestimmte Parameter braucht man den least-squares-Ansatz, um verrauschte Daten auswerten zu können. Für unterbestimmte Modellparameter benötigen wir jedoch **Zusatzinformationen**. Diese werden durch das Konzept der **Regularisierung** in den Invevrsionsprozess eingebracht. Die Stärke der Regularisierung wird durch den $\lambda$-Parameter bestimmt (siehe spalte *lambda* unten bei den Inversionsstatistiken).


## Durchführung der Inversion

* Erstellung eines Modellierung- und Inversionsgitters
* Erstellen eines Vorwärtsmodells
* Erstellen von Messkonfigurationen
* Berechnen synthetischer Daten und verrauschen der Daten
* Vorbereiten der Inversion
* Durchführen der Inversion
* Darstellung/Auswertung der Ergebnisse

## Literatur

* Everett, Inversionskapitel

In [ ]:
import os
import shutil

import crtomo
import reda

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import string
import copy

os.environ["PATH"] += os.pathsep + '/opt/Anaconda3/envs/crtomo/bin'

# Ein synthetisches Beispiel

## Gittererstellung

In [ ]:
import crtomo
grid = crtomo.crt_grid.create_surface_grid(
    nr_electrodes=20, spacing=1, char_lengths=[0.5, 2, 2, 2]
)

grid.plot_grid()

# create the tdManager instance used for the inversion
tdm = crtomo.tdMan(grid=grid)

## Definition Vorwärtsmodell

In [ ]:
# start with a homgeneous model of 1000 Ohm m
pid_mag, _ = tdm.add_homogeneous_model(1000)

In [ ]:
tdm.parman.modify_area(
    pid_mag,
    xmin=0,
    xmax=8,
    zmin=-6,
    zmax=0,
    value=10,
)

fig, axes = tdm.plot_forward_models()
fig.set_figwidth(45 / 2.54)
fig.set_figheight(25 / 2.54)

## Messkonfigurationen

In [ ]:
import reda
configs = reda.ConfigManager(20)
configs.gen_dipole_dipole(skipc=0)

configs.gen_reciprocals(append=True)
# register the configurations with the inversion object
tdm.configs.add_to_configs(configs.configs)

## Vorwärtsmodellierung

In [ ]:
tdm.model()    

In [ ]:
# print generated synthetic measurements
# first column: resistances
# second column: phase values (ignored in this example)
tdm.measurements()

## Verrauschen der synthetischen Daten

In [ ]:
rmag = tdm.measurements()[:, 0]
rpha = tdm.measurements()[:, 1]
# Important: ALWAYS initialize the random number generator using a seed!
np.random.seed(2048)

# absolute component in [Ohm ]
noise_level_rmag_absolute = 0.01
# relative component [0, 1]
noise_level_rmag_relative = 0.15

noise_rmag = np.random.normal(
    loc=0,
    scale=rmag * noise_level_rmag_relative + noise_level_rmag_absolute
)

rmag_with_noise = rmag + noise_rmag

# 0.5 mrad absolute noise level
noise_level_phases = 0.5

noise_rpha = np.random.normal(
    loc=0,
    scale=noise_level_phases
)
rpha_with_noise = rpha + noise_rpha

# register the noise-added data as new measurements and mark them for use in a
# subsequent inversion
tdm.register_measurements(rmag_with_noise, rpha_with_noise)

In [ ]:
indices = np.where(rmag_with_noise <= 0)[0]
tdm.configs.delete_data_points(indices)

## Darstellung der synthetischen Rohdaten in einer Pseudosektion

In [ ]:
ert = reda.ERT()
data = pd.DataFrame(
    np.hstack((
        tdm.configs.configs,
        tdm.measurements()[:, 0, np.newaxis]
    )),
    columns=['a', 'b', 'm', 'n', 'r'],
)
# cast abmn columns to int
for colname in 'abmn':
    data[colname] = data[colname].astype(int)

ert.add_dataframe(data)
fig, ax, cb = ert.pseudosection(log10=True)
fig.set_figwidth(10)
fig.set_figheight(10)

## Durchführen der Inversion

In [ ]:
# Fehlerannahme: relativ 15 %
tdm.crtomo_cfg["mag_rel"] = 15
# Absolute Fehleranname: 0.01 Ohm
tdm.crtomo_cfg["mag_abs"] = 1e-2

tdm.crtomo_cfg["dc_inv"] = "T"
tdm.crtomo_cfg["robust_inv"] = "F"
tdm.crtomo_cfg["diff_inv"] = "F"

In [ ]:
outdir = 'td_syn'
if os.path.isdir(outdir):
    # delete directory
    shutil.rmtree(outdir)
    
# das hier ist die eigentliche Inversion
# Dauert einige Minuten !
tdm.invert(cores=4, output_directory=outdir)

## Darstellung der Ergebnisse

In [ ]:
# compute a plot mask based on the coverage (=cumulated sensitivity), which is a
# weak (!) indicator for data resolution
l1_coverage = tdm.parman.parsets[
    tdm.a['inversion']['l1_dw_log10_norm']
] * 1
abscov = np.abs(l1_coverage)
normcov = np.divide(abscov, 3)
normcov[np.where(normcov > 1)] = 1
mask = np.subtract(1, normcov)

# add this mask to our inversion objectso we can refer to it later
cov_id = tdm.parman.add_data(mask)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

pid_rmag = tdm.a["inversion"]["rmag"][-1]
rmag = tdm.parman.parsets[pid_rmag]

tdm.plot.plot_elements_to_ax(
    cid=pid_rmag,
    # comment out the next line to see the full inversion without an alpha mask
    cid_alpha=cov_id,
    ax=ax,
    plot_colorbar=True,
    cmap_name='jet',
    cblabel=r'$log_{10}(|\rho| [\Omega m])$',
    converter=np.log10,
    no_elecs=True
)
ax.set_title("Inversionsergebnis")
# ax.set_title("mag rel: {}".format(mag_rel))
fig.tight_layout()

# fig.savefig("inversions_ergebnis.jpeg", bbox_inches="tight", dpi=300)

## Kontrolle der RMS-Werte

In [ ]:
# print data misfit for final iteration
tdm.inv_stats.query('type == "main"')

## Aufgaben

* Invertieren sie mit zu kleinen Datenfehlern
* Invertieren sie mit zu großen Datenfehlern

Wie verändern sich die Ergebnisse?